In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/codespace/ann-2022-project
%pwd

Mounted at /content/drive
/content/drive/MyDrive/codespace/ann-2022-project


'/content/drive/MyDrive/codespace/ann-2022-project'

In [6]:
!pip install -q tensorflow-addons
!pip install -q wandb

     |████████████████████████████████| 1.8 MB 6.9 MB/s 
     |████████████████████████████████| 144 kB 57.9 MB/s 
     |████████████████████████████████| 181 kB 71.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [20]:
import tensorflow as tf

from src.dataset import load_test_dataset
from src import Configuration, load_pretrained_googlenet


def probs2label(probs):
    if probs > 0.5:
        label = 1
    else:
        label = 0
    return label


def inference(model, config, col_name):

    tf.random.set_seed(config.seed)

    test_ds, test_df = load_test_dataset(config)

    pred = model.predict(test_ds)

    test_df = test_df.drop("predicted", axis=1)
    test_df[col_name] = list(map(lambda x: probs2label(x), pred))
    test_df["filename"] = test_df["filename"].apply(
        lambda fname: fname.split("./data/test/")[-1]
    )
    return test_df

In [21]:
config = Configuration()
tf.random.set_seed(49)
tf.keras.backend.clear_session()

config.use_augmentation = False
config.use_pretrained = False
config.linear_probing = False
model = load_pretrained_googlenet(config)
model_path = "./models/scratch_noaug"
model.load_weights(model_path)

NANP = inference(model, config, "NA/NP")

Found 67 validated image filenames.


In [22]:
config = Configuration()
tf.random.set_seed(46)
tf.keras.backend.clear_session()

config.use_augmentation = "hard"
config.use_pretrained = False
config.linear_probing = False
model = load_pretrained_googlenet(config)
model_path = "./models/scratch_hardaug"
model.load_weights(model_path)

ANP = inference(model, config, "A/NP")

Found 67 validated image filenames.


In [23]:
config = Configuration()
tf.random.set_seed(46)
tf.keras.backend.clear_session()

config.use_augmentation = False
config.use_pretrained = True
config.linear_probing = False
model = load_pretrained_googlenet(config)
model_path = "./models/finetune_noaug"
model.load_weights(model_path)

NAP = inference(model, config, "NA/P")

Found 67 validated image filenames.


In [24]:
config = Configuration()
tf.random.set_seed(49)
tf.keras.backend.clear_session()

config.use_augmentation = "hard"
config.use_pretrained = True
config.linear_probing = False
model = load_pretrained_googlenet(config)
model_path = "./models/finetune_hardaug"
model.load_weights(model_path)

AP = inference(model, config, "A/P")

Found 67 validated image filenames.


In [27]:
ANP

,filename,A/NP
0,20051020_44338_0100_PP.png,1
1,0656.jpg,1
2,0669.jpg,0
3,20051020_44598_0100_PP.png,0
4,20051020_44284_0100_PP.png,0
...,...,...
62,0672.jpg,1
63,0013.jpg,0
64,214808544484087_1637569673316_RIGHT_4.jpg,1
65,20051020_44692_0100_PP.png,1


In [30]:
from functools import reduce
import pandas as pd

dfs = [NANP, ANP, NAP, AP]
df_final = reduce(lambda left, right: pd.merge(left, right, on='filename'), dfs)

In [32]:
df_final.to_csv("daehyun_submission.csv", index=False)